In [1]:
import re
import pandas as pd
from pathlib import Path
from collections import Counter

pd.set_option('display.max_colwidth', -1)

C:\Users\risto\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


## Data

In [2]:
df=pd.read_csv('data/interim/stenos.csv')

In [3]:
df.shape

(213689, 13)

In [5]:
df.head(1)

,heading,speaker,link_video,link_steno,index_pk,index_snd,text,year,month,day,time,seconds_from_start,video_link_wo_time
0,10:00 Istungi rakendamine,Aseesimees Siim Kallas,https://www.youtube.com/watch?v=GaFL2Zc4OMg&t=0,https://stenogrammid.riigikogu.ee/201911191000,PKP-25040,SND-559399,"Auväärt kolleegid, auväärt Riigikogu, tere hommikust! Kui te nüüd korraks jälle keskendute meie suurepärase istungi rakendamisele ja räägite omavahelisi jutte edasi pärast, siis on mul suur rõõm avada Riigikogu täiskogu II istungjärgu 9. töönädala teisipäevane istung. Kõigepealt on eelnõude ja arupärimiste üleandmine, kui selleks on soovi. Eelnõude ja arupärimiste üleandmise soovi ma ei näe. Siis teeme palun kohaloleku kontrolli. Kohaloleku kontroll Kohal on 75 Riigikogu liiget, puudub 26. Sellest piisab täielikult, et asuda meie päevakorra arutelu juurde.",2019,11,19,10:00,0,https://www.youtube.com/watch?v=GaFL2Zc4OMg


In [6]:
df.text.isnull().sum()

3740

In [7]:
#remove cases where text is missing - might be some data collection errors,
#but not much compared to overall amount of data (less than 2%)
df=df[~pd.isnull(df.text)]
df.shape

(209949, 13)

## See chars we have - is there something funny

In [9]:
cnt_char=Counter(''.join(df.text).lower())

In [10]:
len(cnt_char)

132

In [11]:
cnt_char.most_common(200)

[(' ', 21226757),
 ('e', 16438037),
 ('a', 14882055),
 ('i', 12987380),
 ('s', 11897888),
 ('t', 10191553),
 ('l', 7936321),
 ('u', 7886941),
 ('n', 6457234),
 ('k', 6159097),
 ('o', 5211257),
 ('m', 5122718),
 ('d', 4919949),
 ('r', 3747142),
 ('v', 2819709),
 ('g', 2562789),
 ('p', 2077195),
 ('j', 2046506),
 ('h', 2036389),
 (',', 1981784),
 ('ä', 1793865),
 ('õ', 1532485),
 ('.', 1381396),
 ('ü', 904149),
 ('b', 879255),
 ('ö', 371478),
 ('!', 298545),
 ('\xa0', 226830),
 ('0', 192233),
 ('f', 150215),
 ('1', 147502),
 ('-', 143387),
 ('2', 128476),
 ('?', 107389),
 ('–', 71805),
 ('5', 59881),
 ('3', 52049),
 ('"', 49819),
 (':', 43923),
 ('4', 43464),
 ('6', 37406),
 ('9', 36542),
 ('7', 36345),
 ('8', 33319),
 ('%', 31366),
 ('c', 18931),
 (')', 12687),
 ('(', 12343),
 ('§', 9062),
 ('[', 6624),
 (']', 6623),
 ('y', 5486),
 ('‑', 5269),
 ('š', 4454),
 ('x', 3233),
 ('w', 3056),
 ('ž', 3007),
 ('z', 2427),
 (';', 2355),
 ("'", 1891),
 ('/', 1858),
 ('+', 1154),
 ('q', 308),
 ('&'

## Make text without punctuation (migth be useful for topic modelling)
## Also remove some junk

In [14]:
def remove_punct(text):
    """remove punctuation, junk from text
    -INPUT:
        -text: str of text to be cleaned
    -OUTPUT:
        -cleaned text str"""
    text=re.sub('(?<=[^0-9])\. ', ' ', text) #keep . if they are between characters (no space) or after numbers
    text_clean=text.lower().replace('\x98', ' ').replace('\x9f', ' ').replace('\x84', ' ').replace('\x9b', ' ')\
    .replace('\xad', ' ')\
    .replace('\xa0', ' ').replace('...', ' ')\
    .replace('!',' ').replace('\?',' ').replace('?',' ').replace(', ',' ').replace(')',' ')\
    .replace('(',' ').replace('"',' ')\
    .replace('ºC', ' kraadi ').replace(' – ', ' ').replace(':', ' ').replace('–', ' ').\
    replace('[---]', ' ').replace('[', '').replace(']', '')\
    .replace(';', ' ').replace('+', ' pluss ').replace(' & ', '&').replace('”', ' ').\
    replace('„', ' ').replace('=', ' võrdub ')\
    .replace('´', "'").replace('’', "'").replace('“', ' ').replace(' ±', ' pluss-miinus ').\
    replace(' ..',' ').replace('`', "'")\
    .replace('…', ' ').replace('″', ' ').replace(' = ', ' võrdub ')
    text_clean=' '.join(text_clean.split())
    if text_clean.endswith('.'):
        text_clean=text_clean[:-1]
    return text_clean

remove_punct('Austatud Riigikogu, tere päevast! Teate mis, ')

'austatud riigikogu tere päevast teate mis'

In [15]:
df['text_wo_punct']=df.text.apply(remove_punct)

In [16]:
#we should have less funny characters now
cnt_char_clean=Counter(''.join(df.text_wo_punct).lower())
print(len(cnt_char_clean))
cnt_char_clean.most_common(200)

103


[(' ', 21192736),
 ('e', 16438037),
 ('a', 14882055),
 ('i', 12987394),
 ('s', 11900217),
 ('t', 10191553),
 ('l', 7937482),
 ('u', 7888123),
 ('n', 6457241),
 ('k', 6159097),
 ('o', 5211257),
 ('m', 5122725),
 ('d', 4919963),
 ('r', 3747156),
 ('v', 2819723),
 ('g', 2562789),
 ('p', 2078356),
 ('j', 2046506),
 ('h', 2036389),
 ('ä', 1793865),
 ('õ', 1532499),
 ('ü', 904149),
 ('b', 879269),
 ('ö', 371478),
 ('0', 192233),
 ('f', 150215),
 ('1', 147502),
 ('-', 142779),
 ('2', 128476),
 ('.', 126969),
 ('5', 59881),
 ('3', 52049),
 ('4', 43464),
 ('6', 37406),
 ('9', 36542),
 ('7', 36345),
 ('8', 33319),
 ('%', 31366),
 ('c', 18931),
 (',', 17012),
 ('§', 9062),
 ('y', 5486),
 ('‑', 5269),
 ('š', 4454),
 ('x', 3233),
 ('w', 3056),
 ('ž', 3007),
 ('z', 2427),
 ("'", 1981),
 ('/', 1858),
 ('q', 308),
 ('&', 194),
 ('é', 125),
 ('à', 101),
 ('á', 74),
 ('å', 28),
 ('è', 25),
 ('×', 21),
 ('č', 18),
 ('ó', 16),
 ('@', 14),
 ('ï', 14),
 ('°', 13),
 ('í', 11),
 ('ê', 9),
 ('ç', 8),
 ('²', 7)

## Remove "v a h e a e g"

In [17]:
#not really said but indicates when vaheaeg started
df['text_wo_punct']=df.text_wo_punct.str.replace('v a h e a e g', '')
df['text_wo_punct']=df.text_wo_punct.str.replace('v a h e a eg', '')

## Save data

In [18]:
df.to_csv('data/interim/stenos_cleaned.csv', index=False)